<a href="https://colab.research.google.com/github/Rubenfc1908/Rubenfc1908/blob/main/Mushrooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

instalar PyTorch y torchvision
Configura el entorno para usar una GPU si está disponible.

In [1]:
!pip install torch torchvision

Carga de Datos:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Carga de Imágenes: Lee las imágenes desde los directorios especificados y las almacena en una lista.

Preprocesamiento de Imágenes: Incluye transformaciones como redimensionamiento y normalización, lo cual es crucial para preparar las imágenes para su uso en modelos de aprendizaje profundo.

Carga de Etiquetas: Lee las etiquetas desde los archivos correspondientes en formato de texto, interpretando y almacenando la información necesaria para la detección de objetos.

In [34]:
import cv2
import os
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset  # Agregar esta línea
import cv2
import os
import torchvision.transforms as transforms
import torch
from torch.utils.data import Dataset

# Resto de tu código principal



# Transformación para las imágenes
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_images_and_labels(images_dir, labels_dir):
    images = []
    annotations = []

    # Cargar y preprocesar imágenes
    for img_filename in sorted(os.listdir(images_dir)):
        img_path = os.path.join(images_dir, img_filename)
        img = cv2.imread(img_path)
        img = transform(img)  # Aplicar la transformación
        images.append(img)

        # Leer etiquetas (archivos de texto)
        label_filename = '.'.join(img_filename.split('.')[:-1]) + '.txt'
        label_path = os.path.join(labels_dir, label_filename)

        if os.path.exists(label_path):
            with open(label_path, 'r') as file:
                boxes = []
                max_width = 0
                max_height = 0

                #for line in file:
                    #class_id, x_center, y_center, width, height = map(float, line.split())
                    #print(f"Clase: {class_id}, x_center: {x_center}, y_center: {y_center}, Ancho: {width}, Alto: {height}")

                for line in file:
                    class_id, x_center, y_center, width, height = map(float, line.split())
                    boxes.append((class_id, x_center, y_center, width, height))
                    max_width = max(max_width, width)
                    max_height = max(max_height, height)

                # Aplicar padding para igualar dimensiones
                padded_boxes = []
                for box in boxes:
                    class_id, x_center, y_center, width, height = box
                    width_padding = max_width - width
                    height_padding = max_height - height
                    padded_box = (
                        class_id,
                        x_center,
                        y_center,
                        width + width_padding,
                        height + height_padding,
                    )
                    padded_boxes.append(padded_box)

                annotations.append((img_filename, padded_boxes))

    return images, annotations

# Define la función de collate personalizada
from torch.nn.utils.rnn import pad_sequence
from torch import nn

# Define la función de collate personalizada
from torch.nn.utils.rnn import pad_sequence
from torch import nn



# Rutas a los directorios de imágenes y etiquetas
base_dir = '/content/drive/My Drive/Mushrooms/'

train_images, train_annotations = load_images_and_labels(os.path.join(base_dir, 'train/images'), os.path.join(base_dir, 'train/labels'))
valid_images, valid_annotations = load_images_and_labels(os.path.join(base_dir, 'valid/images'), os.path.join(base_dir, 'valid/labels'))
test_images, test_annotations = load_images_and_labels(os.path.join(base_dir, 'test/images'), os.path.join(base_dir, 'test/labels'))




Saber cuantas clases hay

In [35]:
import os

def count_classes(labels_dir):
    class_ids = set()

    # Iterar sobre cada archivo de etiquetas
    for label_file in os.listdir(labels_dir):
        if label_file.endswith('.txt'):
            with open(os.path.join(labels_dir, label_file), 'r') as file:
                for line in file:
                    class_id = line.split()[0]
                    class_ids.add(class_id)

    return len(class_ids)

# Reemplaza esto con la ruta a tu carpeta de etiquetas
labels_dir = '/content/drive/My Drive/Mushrooms/train/labels/'
num_classes = count_classes(labels_dir)

# Imprime el número de clases (agrega 1 para la clase de fondo)
print("Número de clases, incluyendo el fondo:", num_classes + 1)


Número de clases, incluyendo el fondo: 3


Este código carga un modelo Faster R-CNN preentrenado y luego modifica la última capa de clasificación para que coincida con el número de clases en tu conjunto de datos.

In [36]:
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn

# Cargar el modelo preentrenado
model = fasterrcnn_resnet50_fpn(pretrained=True)

# Si tu conjunto de datos tiene un número diferente de clases, ajusta la cabeza del clasificador
num_classes = 3  # por ejemplo, si tienes dos clases. Asegúrate de incluir la clase de fondo
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Reemplaza la cabeza del clasificador con una nueva para el número de clases en tu conjunto de datos
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

Afinar el modelo R-CNN

In [37]:
# Congelar todas las capas del modelo
for param in model.parameters():
    param.requires_grad = False

# Permitir el entrenamiento de las capas superiores
# Por ejemplo, para la cabeza del clasificador en Faster R-CNN
for param in model.roi_heads.parameters():
    param.requires_grad = True


Dividir tu conjunto de datos en entrenamiento, validación y prueba, y luego crear DataLoaders en PyTorch

In [38]:
import os

images_dir = '/content/drive/My Drive/Mushrooms/train/images/'
labels_dir = '/content/drive/My Drive/Mushrooms/train/labels/'

# Listar todos los archivos de imagen
image_files = set([file.split('.')[0] for file in os.listdir(images_dir)])

# Listar todos los archivos de etiquetas
label_files = set([file.split('.')[0] for file in os.listdir(labels_dir)])

# Encontrar imágenes sin etiquetas correspondientes
unmatched_images = image_files - label_files
unmatched_labels = label_files - image_files

print("Imágenes sin etiquetas:", unmatched_images)
print("Etiquetas sin imágenes:", unmatched_labels)


Imágenes sin etiquetas: set()
Etiquetas sin imágenes: set()


In [39]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        print("Total de imágenes:", len(self.images))
        print("Total de etiquetas:", len(self.labels))

    def __getitem__(self, idx):
        print("Accediendo al índice:", idx)
        return self.images[idx], self.labels[idx]

    def __len__(self):
        return len(self.images)



# Crear conjuntos de datos
train_dataset = CustomDataset(train_images, train_annotations)
valid_dataset = CustomDataset(valid_images, valid_annotations)
test_dataset = CustomDataset(test_images, test_annotations)

# Crear DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Total de imágenes: 246
Total de etiquetas: 246
Total de imágenes: 5
Total de etiquetas: 5
Total de imágenes: 5
Total de etiquetas: 5


Configuración del Proceso de Entrenamiento

In [6]:
import cv2
import os
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset  # Agregar esta línea
import cv2
import os
import torchvision.transforms as transforms
import torch
from torch.utils.data import Dataset

# Resto de tu código principal



# Transformación para las imágenes
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_images_and_labels(images_dir, labels_dir):
    images = []
    targets = []

    # Cargar y preprocesar imágenes y etiquetas
    for img_filename in sorted(os.listdir(images_dir)):
        # Carga de la imagen
        img_path = os.path.join(images_dir, img_filename)
        img = cv2.imread(img_path)
        img = transform(img)
        images.append(img)

        # Preparar las etiquetas
        label_filename = '.'.join(img_filename.split('.')[:-1]) + '.txt'
        label_path = os.path.join(labels_dir, label_filename)

        boxes = []
        labels = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as file:
                for line in file:
                    class_id, x_center, y_center, width, height = map(float, line.split())
                    x_min = x_center - width / 2
                    y_min = y_center - height / 2
                    x_max = x_center + width / 2
                    y_max = y_center + height / 2
                    boxes.append([x_min, y_min, x_max, y_max])
                    labels.append(class_id)

            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
            target = {'boxes': boxes, 'labels': labels}
            targets.append(target)

    return images, targets

def collate_fn(batch):
    images, targets = zip(*batch)
    images = list(images)
    # No intentes apilar los targets. Déjalos como una lista de diccionarios
    return images, list(targets)


# Define la función de collate personalizada
from torch.nn.utils.rnn import pad_sequence
from torch import nn

# Define la función de collate personalizada
from torch.nn.utils.rnn import pad_sequence
from torch import nn



# Rutas a los directorios de imágenes y etiquetas
base_dir = '/content/drive/My Drive/Mushrooms/'

train_images, train_annotations = load_images_and_labels(os.path.join(base_dir, 'train/images'), os.path.join(base_dir, 'train/labels'))
valid_images, valid_annotations = load_images_and_labels(os.path.join(base_dir, 'valid/images'), os.path.join(base_dir, 'valid/labels'))
test_images, test_annotations = load_images_and_labels(os.path.join(base_dir, 'test/images'), os.path.join(base_dir, 'test/labels'))

import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn

# Cargar el modelo preentrenado
model = fasterrcnn_resnet50_fpn(pretrained=True)

# Si tu conjunto de datos tiene un número diferente de clases, ajusta la cabeza del clasificador
num_classes = 3  # por ejemplo, si tienes dos clases. Asegúrate de incluir la clase de fondo
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Reemplaza la cabeza del clasificador con una nueva para el número de clases en tu conjunto de datos
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Congelar todas las capas del modelo
for param in model.parameters():
    param.requires_grad = False

# Permitir el entrenamiento de las capas superiores
# Por ejemplo, para la cabeza del clasificador en Faster R-CNN
for param in model.roi_heads.parameters():
    param.requires_grad = True
import os

images_dir = '/content/drive/My Drive/Mushrooms/train/images/'
labels_dir = '/content/drive/My Drive/Mushrooms/train/labels/'

# Listar todos los archivos de imagen
image_files = set([file.split('.')[0] for file in os.listdir(images_dir)])

# Listar todos los archivos de etiquetas
label_files = set([file.split('.')[0] for file in os.listdir(labels_dir)])

# Encontrar imágenes sin etiquetas correspondientes
unmatched_images = image_files - label_files
unmatched_labels = label_files - image_files

print("Imágenes sin etiquetas:", unmatched_images)
print("Etiquetas sin imágenes:", unmatched_labels)


from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        print("Total de imágenes:", len(self.images))
        print("Total de etiquetas:", len(self.labels))

    def __getitem__(self, idx):
        print("Accediendo al índice:", idx)
        return self.images[idx], self.labels[idx]

    def __len__(self):
        return len(self.images)



# Crear conjuntos de datos
train_dataset = CustomDataset(train_images, train_annotations)
valid_dataset = CustomDataset(valid_images, valid_annotations)
test_dataset = CustomDataset(test_images, test_annotations)

# Crear DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

from torch.nn.utils.rnn import pad_sequence
import torch
import torch.optim as optim

# Definir un optimizador
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Definir la función de pérdida
criterion = torch.nn.CrossEntropyLoss()

# Definir el dispositivo: Utiliza GPU si está disponible, de lo contrario usa CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mover el modelo al dispositivo seleccionado
model = model.to(device)




# Aplicar padding durante la carga de datos
train_images, train_annotations = load_images_and_labels(images_dir, labels_dir)  # Utiliza load_images_and_labels

# Crear un Dataset personalizado
train_dataset = CustomDataset(train_images, train_annotations)

# Usar esta función en tu DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
print
enumerate(train_loader)



class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        assert len(self.images) == len(self.labels), "La cantidad de imágenes y etiquetas debe ser la misma"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if idx < len(self.images) and idx < len(self.labels):
            return self.images[idx], self.labels[idx]
        else:
            raise IndexError("Índice fuera de rango")

num_epochs = 2  # Define aquí el número de épocas---------------------------------------------------------------------------------------------------------------------------------

# Ciclo de entrenamiento
print(f"Longitud de train_loader: {len(train_loader)}")
print(f"Cantidad total de imágenes en entrenamiento: {len(train_images)}")
print(f"Cantidad total de etiquetas en entrenamiento: {len(train_annotations)}")


for epoch in range(num_epochs):
    model.train()  # Poner el modelo en modo de entrenamiento
    train_loss = 0.0

    for images, targets in train_loader:
        images = list(image.to(device) for image in images)  # Mover las imágenes al dispositivo

        # Mover también los targets al dispositivo
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Limpia los gradientes
        optimizer.zero_grad()

        # Forward pass
        loss_dict = model(images, targets)  # El modelo espera una lista de imágenes y una lista de diccionarios de targets

        # Las pérdidas son devueltas como un diccionario. Sumamos todas las pérdidas.
        losses = sum(loss for loss in loss_dict.values())

        # Backward pass y optimizar
        losses.backward()
        optimizer.step()

        train_loss += losses.item()

    # Calcular la pérdida promedio de entrenamiento
    train_loss /= len(train_loader)

    # Resto del código para validación...

  # Validación
    model.eval()
    valid_loss = 0.0

with torch.no_grad():
    for images, targets_batch in valid_loader:
        images = [img.to(device) for img in images]
        processed_targets = []

        for targets in targets_batch:
            if isinstance(targets, dict):
                processed_targets.append({k: v.to(device) for k, v in targets.items()})
            else:
                print("Error: El target no es un diccionario.")

        # Realiza la inferencia
        outputs = model(images, processed_targets)
        # Aquí, puedes calcular la pérdida y acumularla en valid_loss si es necesario

# Calcula la pérdida promedio de validación
valid_loss /= len(valid_loader)

# Imprimir estadísticas de entrenamiento y validación
print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Imágenes sin etiquetas: set()
Etiquetas sin imágenes: set()
Total de imágenes: 246
Total de etiquetas: 246
Total de imágenes: 5
Total de etiquetas: 5
Total de imágenes: 5
Total de etiquetas: 5
Total de imágenes: 246
Total de etiquetas: 246
Longitud de train_loader: 8
Cantidad total de imágenes en entrenamiento: 246
Cantidad total de etiquetas en entrenamiento: 246
Accediendo al índice: 15
Accediendo al índice: 57
Accediendo al índice: 113
Accediendo al índice: 227
Accediendo al índice: 198
Accediendo al índice: 77
Accediendo al índice: 11
Accediendo al índice: 184
Accediendo al índice: 70
Accediendo al índice: 6
Accediendo al índice: 7
Accediendo al índice: 107
Accediendo al índice: 215
Accediendo al índice: 195
Accediendo al índice: 0
Accediendo al índice: 151
Accediendo al índice: 119
Accediendo al índice: 236
Accediendo al índice: 176
Accediendo al índice: 132
Accediendo al índice: 228
Accediendo al índice: 88
Accediendo al índice: 173
Accediendo al índice: 179
Accediendo al índice:

In [ ]:
# Guardar el modelo entrenado
torch.save(model.state_dict(), '/content/drive/My Drive/Mushrooms/modelo_entrenado.pth')


gradio

In [10]:
pip install gradio

In [1]:
!pip install --upgrade typing_extensions

In [2]:
!pip install --force-reinstall gradio

  Using cached gradio-4.9.0-py3-none-any.whl (16.6 MB)
  Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
  Using cached altair-5.2.0-py3-none-any.whl (996 kB)
  Using cached fastapi-0.105.0-py3-none-any.whl (93 kB)
  Using cached ffmpy-0.3.1-py3-none-any.whl
  Using cached gradio_client-0.7.2-py3-none-any.whl (304 kB)
  Using cached httpx-0.25.2-py3-none-any.whl (74 kB)
  Using cached huggingface_hub-0.19.4-py3-none-any.whl (311 kB)
  Using cached importlib_resources-6.1.1-py3-none-any.whl (33 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached MarkupSafe-2.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Using cached matplotlib-3.8.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached numpy-1.26.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached orjson-3.9.10-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (138 kB)
  Using cached packaging-23.2-py3-none-any.

In [11]:
import gradio as gr
import torch
from PIL import Image
import torchvision.transforms as transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import os

# Crear una instancia del modelo y cargar el estado
model = fasterrcnn_resnet50_fpn(pretrained=False, num_classes=3)  # Asegúrate de usar las mismas configuraciones que cuando entrenaste el modelo
model_path = '/content/drive/My Drive/Mushrooms/modelo_entrenado.pth'
model.load_state_dict(torch.load(model_path))
model.eval()

# Definir función para procesar la predicción
def process_prediction(prediction):
    # Mapeo de índices de clases a etiquetas de clase
    class_labels = {1: 'chicken', 2: 'chanterelle'}

    # Obtener las cajas delimitadoras y las etiquetas de clase
    boxes = prediction[0]['boxes'].cpu().numpy()
    labels = prediction[0]['labels'].cpu().numpy()

    # Formatear el resultado
    result = []
    for label, box in zip(labels, boxes):
        if label in class_labels:
            label_str = class_labels[label]
            box_str = f"({box[0]:.2f}, {box[1]:.2f}, {box[2]:.2f}, {box[3]:.2f})"
            result.append(f"{label_str}: {box_str}")

    return result if result else ["No objects detected"]

# Definir función predict
def predict(image):
    # Transformar y preparar la imagen
    transform = transforms.Compose([
        transforms.Resize((416, 416)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    input_tensor = transform(image).unsqueeze(0)

    # Realizar la inferencia
    with torch.no_grad():
        prediction = model(input_tensor)

    # Procesar la predicción
    return process_prediction(prediction)

# Cargar imágenes de prueba
test_images_dir = '/content/drive/My Drive/Mushrooms/test/images/'
test_images = [os.path.join(test_images_dir, file) for file in os.listdir(test_images_dir) if file.endswith('.jpg')]

# Crear la interfaz de Gradio
iface = gr.Interface(fn=predict, inputs="image", outputs="label", examples=test_images)
iface.launch()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b254c7628b71d6d1d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
